In [ ]:
!pip install halo pyarrow numerapi lightgbm tabulate optuna

import os
from halo import Halo
import json
import gc
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import numerapi
import lightgbm
from sklearn.model_selection import KFold
from scipy.stats import spearmanr
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from utils import (
    save_model,
    load_model,
    neutralize,
    get_biggest_change_features,
    get_time_series_cross_val_splits,
    validation_metrics,
    ERA_COL,
    DATA_TYPE_COL,
    TARGET_COL,
    EXAMPLE_PREDS_COL
)

from metrics import evaluate

from numerapi import NumerAPI
napi = NumerAPI()

current_round = napi.get_current_round(tournament=8)  # tournament 8 is the primary Numerai Tournament

# Tournament data changes every week so we specify the round in their name. Training
# and validation data only change periodically, so no need to download them every time.
print('Downloading dataset files...')
napi.download_dataset("numerai_training_data_int8.parquet", "training_data_int8.parquet")
# napi.download_dataset("numerai_tournament_data_int8.parquet", f"tournament_data_int8_{current_round}.parquet")
# napi.download_dataset("numerai_validation_data_int8.parquet", f"validation_data_int8.parquet")
# napi.download_dataset("example_validation_predictions.parquet", "example_validation_predictions.parquet")
napi.download_dataset("features.json", "features.json")

with open("features.json", "r") as f:
    feature_metadata = json.load(f)

features_small = set(feature_metadata["feature_sets"]["small"])
features_medium = set(feature_metadata["feature_sets"]["medium"])

in_medium_not_in_small = features_medium - features_small
features = list(features_small) + list(in_medium_not_in_small)

train_pq_path = "training_data_int8.parquet"

# read in just those features along with era and target columns
read_columns = features + [ERA_COL, DATA_TYPE_COL, TARGET_COL]
df_train = pd.read_parquet(train_pq_path, columns=read_columns)

eras = df_train.era.astype(int)
df_train["era"] = eras

training_data = df_train
gc.collect()

  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 25.6 MB 1.6 MB/s             
     |████████████████████████████████| 2.0 MB 7.1 MB/s            
     |████████████████████████████████| 308 kB 26.4 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 80 kB 19.2 MB/s            
     |████████████████████████████████| 149 kB 25.8 MB/s            
     |████████████████████████████████| 49 kB 29.1 MB/s            
     |████████████████████████████████| 112 kB 6.4 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for halo: filename=halo-0.0.31-py3-none-any.whl size=11261 sha256=98887aed43c84a20a375c812314a6fc2ad082078d2f2d49817c6f72cd730311c
  Stored in directory: /tmp/pip-ephem-wheel-cache-02xg3rdn/wheels/bb/85/47/b7c7338ab52808105f937bd8c04aec5d98a543311ac2c8bed2
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=548da0d4c645e1665ea87a548c29003466

2022-01-16 20:49:23,234 INFO numerapi.utils: target file already exists
2022-01-16 20:49:23,235 INFO numerapi.utils: download complete
2022-01-16 20:49:23,778 INFO numerapi.utils: target file already exists
2022-01-16 20:49:23,779 INFO numerapi.utils: download complete


0

In [ ]:
def era_boosting_train(model, X, y, era_col, proportion, ne, ni):
    features = X.columns
    model.fit(X, y)
    new_df = X.copy()
    new_df[TARGET_COL] = y
    new_df["era"] = era_col

    for i in range(ni-1):
        preds = model.predict(X)
        new_df["pred"] = preds
        era_scores = pd.Series(dtype='float32', index=new_df["era"].unique())
        for era in new_df["era"].unique():
            era_df = new_df[new_df["era"] == era]
            era_scores[era] = spearmanr(era_df["pred"], era_df[TARGET_COL])[0]
        
        era_scores.sort_values(inplace=False)
        worst_eras = era_scores[era_scores <= era_scores.quantile(proportion)].index

        worst_df = new_df[new_df["era"].isin(worst_eras)]
        era_scores.sort_index(inplace=True)
        
        print(f"ne: {model.n_estimators}, iter: {i+1}, mean corr: {np.mean(era_scores)}, sharpe: {np.mean(era_scores)/np.std(era_scores)}")
        
        model.fit(worst_df[features], worst_df[TARGET_COL], init_model = model)
    return model

In [ ]:
# For era boosting, the total number of estimators built will be ne*ni

ni = 9
ne = 250
proportion = 0.5
param_grid = {
    'n_estimators': ne, 
    'learning_rate': 0.003, 
    'num_leaves': 25, 
    'max_depth': 6, 
    'colsample_bytree': 0.1,
    "max_bin": 50,
}

temp_model = lightgbm.LGBMRegressor(**param_grid)
model = era_boosting_train(
    temp_model,
    training_data[features],
    training_data[TARGET_COL],
    era_col=training_data['era'],
    proportion=proportion, 
    ne=ne,
    ni=ni
)

ne: 250, iter: 1, mean corr: 0.06882654875516891, sharpe: 2.3689191621029577
ne: 250, iter: 2, mean corr: 0.07703043520450592, sharpe: 4.084805086898683
ne: 250, iter: 3, mean corr: 0.0848427340388298, sharpe: 5.144846619523415
ne: 250, iter: 4, mean corr: 0.08879073709249496, sharpe: 6.47585416343457
ne: 250, iter: 5, mean corr: 0.09365268051624298, sharpe: 6.301480959693916
ne: 250, iter: 6, mean corr: 0.09600390493869781, sharpe: 7.923205554730569
ne: 250, iter: 7, mean corr: 0.10016566514968872, sharpe: 6.990891605299559
ne: 250, iter: 8, mean corr: 0.10243947803974152, sharpe: 9.374425154250295


In [ ]:
print('Reading minimal features of validation and tournament data...')
validation_data = pd.read_parquet('validation_data_int8.parquet',
                                  columns=read_columns)

print("predicting on validation data")
validation_data.loc[:, "prediction"] = model.predict(
    validation_data.loc[:, features])


Reading minimal features of validation and tournament data...
predicting on validation data


In [ ]:
# compute feature correlations with target
all_feature_corrs = training_data.groupby('era').apply(lambda d: d[features].corrwith(d[TARGET_COL]))
# compute the volatility of the feature correlations
feature_corr_volatility = all_feature_corrs.std()

# calculate the feature exposures of the predictions
feature_exposure_list_val = []
for feature in features:
    feature_exposure_list_val.append(np.corrcoef(validation_data[feature], validation_data["prediction"])[0,1])
feature_exposure_list_val = pd.Series(feature_exposure_list_val, index=features)

riskiest_features = (feature_exposure_list_val.abs()*feature_corr_volatility).sort_values()[-100:].index.tolist()

In [ ]:
feature_exposure_list_val.abs().max()

0.5022857243144736

In [ ]:
# neutralize our predictions to the riskiest features
validation_data[f"preds_neutral_riskiest_50"] = neutralize(
    df=validation_data,
    columns=[f"prediction"],
    neutralizers=riskiest_features,
    proportion=0.75,
    normalize=True,
    era_col=ERA_COL
)


In [ ]:
evaluate(validation_data, ["prediction", "preds_neutral_riskiest_50"])

Metrics for prediction
Spearman Correlation: 0.021
Sharpe Ratio: 0.664
Max Feature Exposure:  0.513
Metrics for preds_neutral_riskiest_50
Spearman Correlation: 0.0224
Sharpe Ratio: 0.942
Max Feature Exposure:  0.1959


,prediction,preds_neutral_riskiest_50
spearman,0.021,0.0224
sharpe,0.664,0.9420
max_feature_exposure,0.513,0.1959


In [ ]:
print('Reading minimal features of tournament data...')
napi.download_dataset("numerai_tournament_data.parquet", f"tournament_data_{current_round}.parquet")

tournament_data = pd.read_parquet(f'tournament_data_{current_round}.parquet',
                                  columns=read_columns)

print("predicting on tournament data")
tournament_data.loc[:, f"prediction"] = model.predict(
    tournament_data.loc[:, features])

predicting on tournament data


In [ ]:
# calculate the feature exposures of the predictions
feature_exposure_list_tourny = []
for feature in features:
    feature_exposure_list_tourny.append(np.corrcoef(tournament_data[feature], tournament_data["prediction"])[0,1])
feature_exposure_list_tourny = pd.Series(feature_exposure_list_tourny, index=features)

riskiest_features = (feature_exposure_list_tourny.abs()*feature_corr_volatility).sort_values()[-100:].index.tolist()

# neutralize our predictions to the riskiest features
tournament_data[f"preds_neutral_riskiest_100"] = neutralize(
    df=tournament_data,
    columns=[f"prediction"],
    neutralizers=riskiest_features,
    proportion=0.75,
    normalize=True,
    era_col=ERA_COL
)


In [ ]:
tournament_data["prediction"] = tournament_data["preds_neutral_riskiest_100"].rank(pct=True)
tournament_data["prediction"].to_csv(f"tournament_predictions_{current_round}.csv")